In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch.utils.data as data
import torch_geometric
from torch_geometric.datasets import WebKB
from torch_geometric.utils import to_networkx, k_hop_subgraph

import networkx as nx

In [2]:
dataset = WebKB(root="/home/siddy/META/data", name='cornell')
data = dataset[0]

In [3]:
data

Data(x=[183, 1703], edge_index=[2, 298], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10])

In [4]:
#netwrokx graph G
G = to_networkx(data, to_undirected=True)

In [5]:
def init_ball(radius: int, graph):
    edge_dict = {}
    for index, node in enumerate(graph.nodes()):
        paths = nx.single_source_shortest_path(graph, node, radius)
        if index not in edge_dict:
            edge_dict[index] = []
        for key, value in paths.items():
            if len(value) == 2:
                edge_dict[index].append(value)
            elif len(value)==3:
                edge_dict[index].append(value[1:])
    return edge_dict

In [6]:
edge_dict= init_ball(radius=2, graph=G)

In [ ]:
edge_attn = Edge_atten(in_channels=data.x.size(1), out_channels=64)
ball_gcn = BallGCN(in_channels=data.x.size(1), hidden_channels=64)

In [16]:
optimizer = torch.optim.Adam(set(edge_attn.parameters())| set(ball_gcn.parameters()), lr=0.001, weight_decay=5e-4)

NameError: name 'edge_attn' is not defined

In [ ]:
def train():
    edge_attn.train()
    ball_gcn.train()
    out = ball_gcn(data.x, data.edge_index, data.edge_attr)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return float(loss)

In [ ]:
@torch.no_grad()
def test():
    edge_attn.eval()
    ball_gcn.eval()
    pred = model(data.x, data.edge_index, data.edge_attr).argmax(dim=-1)

    accs=[]
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        accs.append(int((pred[mask] == data.y[mask]).sum()) / int(mask.sum()))
    return accs

In [ ]:
best_val_acc = final_test_acc = 0
times = []
for epoch in range(1, epochs+1):
    start = time.time()
    loss= train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    print(f"Epoch:{epoch}, Loss:{loss}, Train:{train_acc}, Val:{val_acc}, Test:{test_acc}")
    times.append(time.time()-start)
print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")

In [ ]:
b

import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from math_utils import acos, arcosh

class ProductDistance():
    def __init__(self, x, y, k, atten_prob):
        self.x = torch.tensor(x)
        #print(x.dtype)
        self.y = torch.tensor(y)
        self.k = torch.tensor(k)
        self.atten_prob = atten_prob
        self.eps = {torch.float32: 1e-7, torch.float64: 1e-15}
    
    def poincaresqdist(self):
        if len(self.x) != len(self.y):
            raise ValueError("Input vectors must have the same dimensions")
        
        K = 1./ torch.sqrt(- self.k)
        prod = self.atten_prob**2
        #dist = torch.clamp(1. - ((2*self.k*(prod))/((1+self.k*(torch.norm(self.x, p=2.0, dtype=torch.float)**2))*(1+self.k*(torch.norm(self.y, p=2.0, dtype=torch.float)**2)))), min=1.0 + self.eps[self.x.dtype])
        dist = 1. - ((2*self.k*(prod))/((1+self.k*(torch.norm(self.x, p=2.0, dtype=torch.float)**2))*(1+self.k*(torch.norm(self.y, p=2.0, dtype=torch.float)**2))))
        sqdist = (K * arcosh(dist))**2
        return sqdist

    def hyperspheresqdist(self):
        if len(self.x) != len(self.y):
            raise ValueError("Input vectors must have the same dimensions")
        
        K = 1./ torch.sqrt(self.k)
        prod = self.atten_prob**2
        #dist = torch.clamp(1. - ((2*self.k*(prod))/((1+self.k*(torch.norm(self.x, p=2, dtype=torch.float)**2))*(1+self.k*(torch.norm(self.y, p=2, dtype=torch.float)**2)))), min=1.0 + self.eps[self.x.dtype])
        dist = 1. - ((2*self.k*(prod))/((1+self.k*(torch.norm(self.x, p=2, dtype=torch.float)**2))*(1+self.k*(torch.norm(self.y, p=2, dtype=torch.float)**2))))     
        sqdist = (K * acos(dist))**2
        return sqdist

    def euclideansqdist(self):
        if len(self.x) != len(self.y):
            raise ValueError("Input vectors must have the same dimensions")

        dist = torch.norm(self.x - self.y, p=2, dtype=torch.float)
        sqdist = dist**2
        return sqdist

    def productdistance(self):
        poincare_dist = self.poincaresqdist()
        hypersphere_dist = self.hyperspheresqdist()
        euclidean_dist = self.euclideansqdist()

        return torch.sqrt(poincare_dist+hypersphere_dist+euclidean_dist)


if __name__ == "__main__":
    k = 1.0
    x = [1, 2, 3]
    y = [4, 5, 6]


    product_distance_calculator = ProductDistance(x, y, k, atten_prob=1)
    print("Poincare Distance:", product_distance_calculator.poincaresqdist())
    print("Hypersphere Distance:", product_distance_calculator.hyperspheresqdist())
    print("Euclidean Distance:", product_distance_calculator.euclideansqdist())
    print("Product Distance:", product_distance_calculator.productdistance())